In [70]:
import os
import numpy as np
import fitz
import pandas as pd
from tqdm import tqdm
from numba import jit
fol_path = r'\\isco-pipe.local\root\Common\EPC\Crowder Industrial\Crowder Santee Cooper Cross ELG\Isometrics'

In [71]:
def extract_text_pymupdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

@jit(nopython=True)
def do_extraction(full_pdf_path):
    # Extract text using PyMuPDF
    pymupdf_text = extract_text_pymupdf(full_pdf_path).split("\n")
    bill_of_materials = []
    start_collecting = False
    for line in pymupdf_text:
        if line=='N':
            break
        if line=='BILL OF MATERIALS':
            start_collecting = True
        bill_of_materials.append(line)
    return bill_of_materials

@jit(nopython=True)
def handle_missing_values(table,gap,expected_length):
    fixed_new_table = []
    last_id = 0
    last_id_pos = 0
    for el_pos,element in enumerate(table):
        if element.strip().isnumeric():
            int_el = int(element)
            if last_id_pos==0:
                last_id = int_el
                last_id_pos = el_pos
            elif int_el == last_id + 1:
                if el_pos==last_id_pos + gap:# and el_pos>last_id_pos+3:
                    last_id = int_el
                    last_id_pos = el_pos
                    fixed_new_table.append('')
        fixed_new_table.append(element)
    if len(fixed_new_table)%expected_length == gap:
        fixed_new_table.append('')
    try:
        return np.array(fixed_new_table).reshape(-1,expected_length)
    except:
        print(fixed_new_table)
@jit(nopython=True)
def make_table(bill_of_materials):
    new_table = []
    second_table = []
    third_table = []
    unallowed = ['BILL OF MATERIALS','PIPE','FITTINGS','OLETS','FLANGES','VALVES','PIPE SUPPORTS']
    columns = ['ID','QTY','ND','DESCRIPTION','SUPPORT DETAIL','INSTRUMENTS']
    columns_seen = [0,0,0,0,0,0]
    table_to_input = new_table
    second_table_used = False
    third_table_used = False
    
    for t_element in bill_of_materials:
        if not t_element.upper().strip() in unallowed:
            try:
                col_i = columns.index(t_element.upper().strip())
            except:
                col_i = -1
            if col_i!=-1:
                if columns_seen[col_i]==0:
                    if t_element=='SUPPORT DETAIL':
                        new_table = table_to_input
                        table_to_input = second_table
                        table_to_input.extend(columns[:-2])
                        second_table_used = True
                    elif t_element=='INSTRUMENTS':
                        second_table = table_to_input
                        table_to_input = third_table
                        table_to_input.extend(columns[:-3])
                        third_table_used = True
                    columns_seen[col_i]=1
                elif columns_seen[col_i]==1:
                    continue
            table_to_input.append(t_element)
    if third_table_used:
        third_table = table_to_input
    elif second_table_used:
        second_table = table_to_input
    else:
        new_table = table_to_input
    #handle errors where the last value in a row is blank by keeping track of the IDs (each line is prev_ID + 1) 
    #and making sure that there are at least 3 values between them, if not, add an empty string
    
#     new_table = np.array(fixed_new_table).reshape(-1,4)
# #     except:
# #         print('new:',fixed_new_table)
    
#     if len(second_table)>0:
#         #print(second_table)
#         fixed_second_table = []
#         last_id = 0
#         last_id_pos = 0
#         for el_pos,element in enumerate(second_table):
#             if element.strip().isnumeric():
#                 int_el = int(element)
#                 if last_id_pos==0:
#                     last_id = int_el
#                     last_id_pos = el_pos
#                 elif int_el == last_id + 1:
#                     if el_pos==last_id_pos + 4:# and el_pos>last_id_pos:
#                         last_id = int_el
#                         last_id_pos = el_pos
#                         fixed_second_table.append('')
#             fixed_second_table.append(element)
# #         try:
#         if len(fixed_second_table)%5 == 4:
#             fixed_second_table.append('')
#         second_table = np.array(fixed_second_table).reshape(-1,5)
#         except:
#             print('second:',fixed_second_table)
    new_table=handle_missing_values(new_table,3,4)
    if second_table:
        second_table=handle_missing_values(second_table,4,5)
    if third_table:
        third_table=handle_missing_values(third_table,3,4)
    return new_table,second_table,third_table
@jit(nopython=True)
def combine_tables(new_table,second_table,third_table,pdf_path):
    ndf = pd.DataFrame(new_table)
    ndf.columns = ndf.iloc[0]
    ndf = ndf.drop(0)
    if isinstance(second_table,np.ndarray):
        sdf = pd.DataFrame(second_table)
        sdf.columns = sdf.iloc[0]
        sdf = sdf.drop(0)
        combined = pd.concat([ndf, sdf])#.fillna(None)
        final = combined
    else:
        final = ndf
        
    if isinstance(third_table,np.ndarray):
        sdf = pd.DataFrame(third_table)
        sdf.columns = sdf.iloc[0]
        sdf = sdf.drop(0)
        combined = pd.concat([final, sdf])#.fillna(None)
        final = combined
    else:
        final = ndf
    final['FILE_PATH'] = os.path.split(pdf_path)[1]
    return final
@jit(nopython=True)
def extract_bill_of_materials(pdf_path):
    bill_of_materials = do_extraction(pdf_path)
    t1,t2,t3 = make_table(bill_of_materials)
    t = combine_tables(t1,t2,t3,pdf_path)
    return t
@jit(nopython=True)
def extract_bills_of_materials(fol_path):
    bills_of_materials = []
    errors = []
    for pdf in tqdm(os.listdir(fol_path)):
        #print(pdf)
        if not os.path.splitext(pdf)[1]=='.pdf':continue
        pdf_path = os.path.join(fol_path,pdf)
        try:
            bills_of_materials.append(extract_bill_of_materials(pdf_path))
        except Exception as e:
            print((pdf_path,e))
            errors.append((pdf_path,e))
            
    return bills_of_materials,errors

In [72]:
bills_of_materials,errors = extract_bills_of_materials(fol_path)

 61%|████████████████████████████████████████████████▌                               | 209/344 [03:00<01:59,  1.13it/s]

['ID', 'QTY', 'ND', 'DESCRIPTION', '1', '24\'-6"', '6"', 'PIPE, IPS DR 11, ASTM D3350, PE 4710', '2', '5', '6"', 'ELL 90, HDPE, PE, IPS DR 11, MOLDED', '3', '1', '6"', 'FLANGE ADAPTER, IPS DR 11, LJXPE, ASTM F2880. PE4710,  WITH BACK-UP RING, CL 150, DR 11,', 'ASME/ANSI 16.5']
('\\\\isco-pipe.local\\root\\Common\\EPC\\Crowder Industrial\\Crowder Santee Cooper Cross ELG\\Isometrics\\6-0-528-40262-PE01 - Isometric.pdf', IndexError('single positional indexer is out-of-bounds'))


100%|████████████████████████████████████████████████████████████████████████████████| 344/344 [04:55<00:00,  1.17it/s]


In [73]:
errors

[('\\\\isco-pipe.local\\root\\Common\\EPC\\Crowder Industrial\\Crowder Santee Cooper Cross ELG\\Isometrics\\6-0-528-40262-PE01 - Isometric.pdf',
  IndexError('single positional indexer is out-of-bounds'))]

In [74]:
df = pd.concat(bills_of_materials)
len(df['FILE_PATH'].unique())

341

In [75]:
df.to_excel(os.path.join(fol_path,'extracted_boms_fixed.xlsx'))

In [50]:
with open(os.path.join(fol_path,'extracted_boms_errors.txt'),'w') as error_txt:
    for error in errors:
        error_txt.write(str(error)+'\n')

In [81]:
from IPython.core.display import HTML
a=np.array(['ID', 'QTY', 'ND', 'DESCRIPTION', '1', '24\'-6"', '6"', 'PIPE, IPS DR 11, ASTM D3350, PE 4710', '2', '5', '6"', 'ELL 90, HDPE, PE, IPS DR 11, MOLDED', '3', '1', '6"', 'FLANGE ADAPTER, IPS DR 11, LJXPE, ASTM F2880. PE4710,  WITH BACK-UP RING, CL 150, DR 11,ASME/ANSI 16.5'])
HTML(pd.DataFrame(a.reshape(-1,4)).to_html())

,0,1,2,3
0,ID,QTY,ND,DESCRIPTION
1,1,"24'-6""","6""","PIPE, IPS DR 11, ASTM D3350, PE 4710"
2,2,5,"6""","ELL 90, HDPE, PE, IPS DR 11, MOLDED"
3,3,1,"6""","FLANGE ADAPTER, IPS DR 11, LJXPE, ASTM F2880. PE4710, WITH BACK-UP RING, CL 150, DR 11,ASME/ANSI 16.5"
